In [7]:
import numpy as np
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt

In [ ]:
# Define the current GSD and desired GSD values
directory = '/work/scorreacardo_umass_edu/DeepSatGSD/data/interim/GSD_50cm'
current_gsd = 0.5  # meters per pixel

#0.75m, 1m, 1.25m, 1.5m, and 2m
desired_gsd_values = [0.80, 1.0, 1.5, 1.75, 2.0, 2.5, 3]  # meters per pixel
dic_filename_gsd_values = {0.80:"GSD_80cm",
                          1.0:"GSD_100cm",
                          1.5:"GSD_150cm",
                          1.75:"GSD_175cm",
                          2.0:"GSD_200cm",
                          2.5:"GSD_250cm",
                          3:"GSD_300cm"}
counter = 0
for filename in os.listdir(directory):
    counter += 1
    print(f"This is image number: {counter}")
    image_path = os.path.join(directory, filename)
    # Open the original image
    with rasterio.open(image_path) as src:
        # Read the image data
        image_data = src.read()

        # Calculate the resampling factors
        resampling_factors = [current_gsd / desired_gsd for desired_gsd in desired_gsd_values]

        # Resample the image for each desired GSD
        for i, factor in enumerate(resampling_factors):
            # Compute the new image size based on the resampling factor
            new_height = int(src.height * factor)
            #print(f"new height: {new_height}")
            new_width = int(src.width * factor)
            #print(f"new width: {new_width}")

            # Resample the image using the desired GSD
            resampled_data = src.read(
                out_shape=(src.count, new_height, new_width),
                resampling=Resampling.bilinear
            )

            # Create a new TIF file name based on the desired GSD
            new_image_path = f"resampled_image_{new_gsd}m.tif"

            # Update metadata for the resampled image
            new_meta = src.meta.copy()
            new_meta.update({
                'width': new_width,
                'height': new_height,
                #'transform': src.transform * src.transform.scale(pixel_factor, pixel_factor)
            })

            # Save the resampled image
            with rasterio.open(new_image_path, 'w', **new_meta) as dst:
                dst.write(resampled_data)
            # Open the original image
            with rasterio.open(new_image_path) as dst:
                # Read the image data
                image_data = dst.read()
                new_gsd = desired_gsd_values[i]
                pixel_factor = current_gsd / new_gsd
                new_height = int(dst.height / pixel_factor)
                print(f"new height: {new_height}")
                new_width = int(dst.width / pixel_factor)
                print(f"new width: {new_width}")
                # Resample the image using the desired GSD
                new_resampled_data = dst.read(
                    out_shape=(dst.count, new_height, new_width),
                    resampling=Resampling.bilinear
                )

                # Create a new TIF file name based on the desired GSD
                final_image_path = f"/work/scorreacardo_umass_edu/DeepSatGSD/data/interim/{dic_filename_gsd_values[desired_gsd_values[i]]}/{filename[:-4]}_resampled_image_{new_gsd}m.tif"

                # Update metadata for the resampled image
                final_meta = dst.meta.copy()
                final_meta.update({
                    'width': new_width,
                    'height': new_height,
                    #'transform': dst.transform * dst.transform.scale(pixel_factor, pixel_factor)
                })
                # Save the resampled image
                with rasterio.open(final_image_path, 'w', **final_meta) as final_dst:
                    final_dst.write(new_resampled_data)